# HuBMAP - Exploratory Data Analysis

HuBMAP에 대한 데이터 분석: 신장 문제 해결 - [HuBMAP: Hacking the Kidney](https://www.kaggle.com/c/hubmap-kidney-segmentation)

이 해커톤에 사용된 HuBMAP 데이터에는 11개의 frozen frozen 및 9개의 FFPE(Formalin Fixed Paraffin Embedded) PAS 신장 이미지가 포함됩니다. 글로머룰리 FTU 주석이 20개 조직 샘플 모두에 대해 존재합니다. 이 중 일부는 교육을 위해 공유되고 다른 일부는 제출하는 데 사용됩니다.

![](https://storage.googleapis.com/kaggle-competitions/kaggle/22990/logos/header.png)

<a id="top"></a>

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" style='color:white; background:#EAA6D1; border:0' role="tab" aria-controls="home"><center>목차</center></h3>

* [1. 간단한 데이터 탐색](#1)
* [2. 이미지 및 마스크 시각화](#2)
* [3. 메타데이터 분석](#3)

In [1]:
!pip install -q -U pip
!pip install -q -U seaborn

<a id="1"></a>
<h2 style='background:#EAA6D1; border:0; color:white'><center>간단한 데이터 탐색<center><h2>

In [2]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import cv2
import tifffile

In [3]:
BASE_PATH = "../input/hubmap-kidney-segmentation/"
TRAIN_PATH = os.path.join(BASE_PATH, "train")

print(os.listdir(BASE_PATH))

['sample_submission.csv', 'HuBMAP-20-dataset_information.csv', 'train.csv', 'test', 'train']


### Train masks (학습 마스크)

**train.csv** 에는 각 이미지에 대한 고유 ID와 함께 이미지의 개체에 대한 마스크의 RLE 인코딩 표현이 포함됩니다. RLE 인코딩 방식에 대한 자세한 내용은 evaluation 정보를 참조하세요.

In [4]:
df_train = pd.read_csv(
    os.path.join(BASE_PATH, "train.csv")
)
df_train

,id,encoding
0,2f6ecfcdf,296084587 4 296115835 6 296115859 14 296147109...
1,aaa6a05cc,30989109 59 31007591 64 31026074 68 31044556 7...
2,cb2d976f4,78144363 5 78179297 15 78214231 25 78249165 35...
3,0486052bb,101676003 6 101701785 8 101727568 9 101753351 ...
4,e79de561c,7464094 14 7480273 41 7496453 67 7512632 82 75...
5,095bf7a1f,113430380 22 113468538 67 113506697 111 113544...
6,54f2eec69,124601765 36 124632133 109 124662536 147 12469...
7,1e2425f28,49453112 7 49479881 22 49506657 31 49533433 40...


### Submission df

In [5]:
df_sub = pd.read_csv(
    os.path.join(BASE_PATH, "sample_submission.csv"))
df_sub

,id,predicted
0,b9a3865fc,NaN
1,b2dc8411c,NaN
2,26dc41664,NaN
3,c68fe75ea,NaN
4,afa5e8098,NaN


### Number of samples

In [6]:
print(f"Number of train images: {df_train.shape[0]}")
print(f"Number of test images: {df_sub.shape[0]}")

Number of train images: 8
Number of test images: 5


### Train and test metadata

**HuBMAP-20-dataset_information.csv** 에는 각 이미지에 대한 추가 정보(익명 환자 데이터를 포함)가 있습니다.

In [7]:
df_info = pd.read_csv(
    os.path.join(BASE_PATH, "HuBMAP-20-dataset_information.csv")
)
df_info.sample(3)

,image_file,width_pixels,height_pixels,anatomical_structures_segmention_file,glomerulus_segmentation_file,patient_number,race,ethnicity,sex,age,weight_kilograms,height_centimeters,bmi_kg/m^2,laterality,percent_cortex,percent_medulla
8,b9a3865fc.tiff,40429,31295,b9a3865fc-anatomical-structure.json,b9a3865fc.json,67347,White,Not Hispanic or Latino,Female,58,59.0,160.0,23.0,Right,55,45
11,0486052bb.tiff,34937,25784,0486052bb-anatomical-structure.json,0486052bb.json,67177,White,Not Hispanic or Latino,Male,31,106.1,180.3,32.6,Right,80,20
4,c68fe75ea.tiff,19780,26840,c68fe75ea-anatomical-structure.json,c68fe75ea.json,67112,White,Not Hispanic or Latino,Male,56,91.2,167.6,32.5,Left,80,20


## Utility functions

In [8]:
# https://www.kaggle.com/paulorzp/rle-functions-run-lenght-encode-decode
# RLE 인코딩을 Mask 이미지로 변환
def rle2mask(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [
        np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])
    ]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = 1
    return img.reshape(shape).T


# 이미지 및 마스크 불러오기
def read_image(image_id, scale=None, verbose=1):
    image = tifffile.imread(
        os.path.join(BASE_PATH, f"train/{image_id}.tiff")
    )
    if len(image.shape) == 5:
        image = image.squeeze().transpose(1, 2, 0)
    
    mask = rle2mask(
        df_train[df_train["id"] == image_id]["encoding"].values[0], 
        (image.shape[1], image.shape[0])
    )
    
    if verbose:
        print(f"[{image_id}] Image shape: {image.shape}")
        print(f"[{image_id}] Mask shape: {mask.shape}")
    
    if scale:
        new_size = (image.shape[1] // scale, image.shape[0] // scale)
        image = cv2.resize(image, new_size)
        mask = cv2.resize(mask, new_size)
        
        if verbose:
            print(f"[{image_id}] Resized Image shape: {image.shape}")
            print(f"[{image_id}] Resized Mask shape: {mask.shape}")
        
    return image, mask


# 테스트 이미지 불러오기 (마스크 제외)
def read_test_image(image_id, scale=None, verbose=1):
    image = tifffile.imread(
        os.path.join(BASE_PATH, f"test/{image_id}.tiff")
    )
    if len(image.shape) == 5:
        image = image.squeeze().transpose(1, 2, 0)
    
    if verbose:
        print(f"[{image_id}] Image shape: {image.shape}")
    
    if scale:
        new_size = (image.shape[1] // scale, image.shape[0] // scale)
        image = cv2.resize(image, new_size)
        
        if verbose:
            print(f"[{image_id}] Resized Image shape: {image.shape}")
        
    return image


# 이미지 및 마스크 시각화
def plot_image_and_mask(image, mask, image_id):
    plt.figure(figsize=(16, 10))
    
    plt.subplot(1, 3, 1)
    plt.imshow(image)
    plt.title(f"Image {image_id}", fontsize=18)
    
    plt.subplot(1, 3, 2)
    plt.imshow(image)
    plt.imshow(mask, cmap="hot", alpha=0.5)
    plt.title(f"Image {image_id} + mask", fontsize=18)    
    
    plt.subplot(1, 3, 3)
    plt.imshow(mask, cmap="hot")
    plt.title(f"Mask", fontsize=18)    
    
    plt.show()
    

# 이미지 및 마스크 정사각형으로 불러오기 
def plot_grid_image_with_mask(image, mask):
    plt.figure(figsize=(16, 16))
    
    w_len = image.shape[0]
    h_len = image.shape[1]
    
    min_len = min(w_len, h_len)
    w_start = (w_len - min_len) // 2
    h_start = (h_len - min_len) // 2
    
    plt.imshow(image[w_start : w_start + min_len, h_start : h_start + min_len])
    plt.imshow(
        mask[w_start : w_start + min_len, h_start : h_start + min_len], cmap="hot", alpha=0.5,
    )
    plt.axis("off")
            
    plt.show()
    

# Whole Slide 이미지에서 패치만 불러오기
def plot_slice_image_and_mask(image, mask, start_h, end_h, start_w, end_w):
    plt.figure(figsize=(16, 5))
    
    sub_image = image[start_h:end_h, start_w:end_w, :]
    sub_mask = mask[start_h:end_h, start_w:end_w]
    
    plt.subplot(1, 3, 1)
    plt.imshow(sub_image)
    plt.axis("off")
    
    plt.subplot(1, 3, 2)
    plt.imshow(sub_image)
    plt.imshow(sub_mask, cmap="hot", alpha=0.5)
    plt.axis("off")
    
    plt.subplot(1, 3, 3)
    plt.imshow(sub_mask, cmap="hot")
    plt.axis("off")
    
    plt.show()

<a id="2"></a>
<h2 style='background:#EAA6D1; border:0; color:white'><center>이미지 및 마스크 시각화<center><h2>

In [ ]:
small_ids = [
    "0486052bb", "095bf7a1f", "1e2425f28", "2f6ecfcdf",
    "54f2eec69", "aaa6a05cc", "cb2d976f4", "e79de561c",
]
small_images = []
small_masks = []

for small_id in small_ids:
    tmp_image, tmp_mask = read_image(small_id, scale=20, verbose=0)
    small_images.append(tmp_image)
    small_masks.append(tmp_mask)

## Train images

In [ ]:
plt.figure(figsize=(16, 16))
for ind, (tmp_id, tmp_image) in enumerate(zip(small_ids, small_images)):
    plt.subplot(3, 3, ind + 1)
    plt.imshow(tmp_image)
    plt.axis("off")

## Train images + masks

In [ ]:
plt.figure(figsize=(16, 16))
for ind, (tmp_id, tmp_image, tmp_mask) in enumerate(zip(small_ids, small_images, small_masks)):
    plt.subplot(3, 3, ind + 1)
    plt.imshow(tmp_image)
    plt.imshow(tmp_mask, cmap="hot", alpha=0.5)
    plt.axis("off")

In [ ]:
small_ids = [
    "26dc41664", "afa5e8098", "b2dc8411c", "b9a3865fc", "c68fe75ea",
]
small_images = []

for small_id in small_ids:
    tmp_image = read_test_image(small_id, scale=20, verbose=0)
    small_images.append(tmp_image)

## Test images

In [ ]:
plt.figure(figsize=(16, 11))
for ind, (tmp_id, tmp_image) in enumerate(zip(small_ids, small_images)):
    plt.subplot(2, 3, ind + 1)
    plt.imshow(tmp_image)
    plt.axis("off")

## 0486052bb

In [ ]:
image_id = "0486052bb"
image, mask = read_image(image_id, 2)

In [ ]:
plot_image_and_mask(image, mask, image_id)

In [ ]:
plot_slice_image_and_mask(image, mask, 5000, 7500, 2500, 5000)
plot_slice_image_and_mask(image, mask, 5250, 5720, 3500, 4000)
plot_slice_image_and_mask(image, mask, 5375, 5575, 3650, 3850)

In [ ]:
plot_grid_image_with_mask(image, mask)

## 095bf7a1f

In [ ]:
image_id = "095bf7a1f"
image, mask = read_image(image_id, scale=2)

In [ ]:
plot_image_and_mask(image, mask, image_id)

In [ ]:
plot_slice_image_and_mask(image, mask, 7500, 10000, 10000, 12500)

In [ ]:
plot_grid_image_with_mask(image, mask)

## 1e2425f28

In [ ]:
image_id = "1e2425f28"
image, mask = read_image(image_id, scale=2)

In [ ]:
plot_image_and_mask(image, mask, image_id)

## 2f6ecfcdf

In [ ]:
image_id = "2f6ecfcdf"
image, mask = read_image(image_id, scale=2)

In [ ]:
plot_image_and_mask(image, mask, image_id)

In [ ]:
plot_slice_image_and_mask(image, mask, 10000, 12000, 8000, 10000)

## aaa6a05cc

In [ ]:
image_id = "aaa6a05cc"
image, mask = read_image(image_id)

In [ ]:
plot_image_and_mask(image, mask, image_id)

In [ ]:
plot_slice_image_and_mask(image, mask, 6500, 8500, 7000, 9000)

## e79de561c

In [ ]:
image_id = "e79de561c"
image, mask = read_image(image_id)

In [ ]:
plot_image_and_mask(image, mask, image_id)

In [ ]:
plot_slice_image_and_mask(image, mask, 4000, 6000, 2000, 4000)

<a id="3"></a>
<h2 style='background:#EAA6D1; border:0; color:white'><center>Metadata Analysis<center><h2>

In [ ]:
pd.read_json(
    os.path.join(BASE_PATH, "train/0486052bb-anatomical-structure.json")
)

In [ ]:
pd.read_json(
    os.path.join(BASE_PATH, "train/0486052bb.json")
)

In [ ]:
df_info["split"] = "test"
df_info.loc[df_info["image_file"].isin(os.listdir(os.path.join(BASE_PATH, "train"))), "split"] = "train"

In [ ]:
df_info["area"] = df_info["width_pixels"] * df_info["height_pixels"]

In [ ]:
df_info.head()

In [ ]:
plt.figure(figsize=(16, 35))
plt.subplot(6, 2, 1)
sn.countplot(x="race", hue="split", data=df_info)
plt.subplot(6, 2, 2)
sn.countplot(x="ethnicity", hue="split", data=df_info)
plt.subplot(6, 2, 3)
sn.countplot(x="sex", hue="split", data=df_info)
plt.subplot(6, 2, 4)
sn.countplot(x="laterality", hue="split", data=df_info)
plt.subplot(6, 2, 5)
sn.histplot(x="age", hue="split", data=df_info)
plt.subplot(6, 2, 6)
sn.histplot(x="weight_kilograms", hue="split", data=df_info)
plt.subplot(6, 2, 7)
sn.histplot(x="height_centimeters", hue="split", data=df_info)
plt.subplot(6, 2, 8)
sn.histplot(x="bmi_kg/m^2", hue="split", data=df_info)
plt.subplot(6, 2, 9)
sn.histplot(x="percent_cortex", hue="split", data=df_info)
plt.subplot(6, 2, 10)
sn.histplot(x="percent_medulla", hue="split", data=df_info)
plt.subplot(6, 2, 11)
sn.histplot(x="area", hue="split", data=df_info);

# WORK IN PROGRESS...